In [1]:
import pandas as pd
import datetime, pytz
import vectorbt as vbt
import numpy as np
import matplotlib.pyplot as plt
import json

import talib
import pandas_ta
import time

from importlib import reload


import sys 
sys.path.append('../../modules/')

from forwardtest import ForwardTest
from vbt_indicator import Indicators



c:\Users\slavi\anaconda3\envs\Bruno_proj\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:
def triplestrend(High, Low, Close, ema_window = 90, stl1 =20, stm1 = 3, stl2 = 20, stm2 = 4, stl3 = 40, stm3 = 8):
    ema = vbt.talib('EMA').run(Close, ema_window).real.to_numpy()
    stoch_rsi = vbt.talib('STOCHRSI').run(Close).fastk.to_numpy() 
    st1 = vbt.pandas_ta('supertrend').run(High, Low, Close, stl1, stm1).supertd.to_numpy()
    st2 = vbt.pandas_ta('supertrend').run(High, Low, Close, stl2, stm2).supertd.to_numpy()
    st3 = vbt.pandas_ta('supertrend').run(High, Low, Close, stl3, stm3).supertd.to_numpy()
    
    entries = ((st1 + st2 + st3)>= 1) & (stoch_rsi > 80) & (ema > Close)
    exits =  ((st1 + st2 + st3) < 1) & (stoch_rsi < 20) & (ema < Close)

    return entries, exits


TripleSTrend = vbt.IndicatorFactory(
    class_name= 'TripleSTrend',
    short_name = '3ST',
    input_names = ["High", "Low", "Close"] ,
    param_names = ["ema_window", "stl1", "stm1", "stl2", "stm2", "stl3", "stm3"],
    output_names= ["entries", "exits"]
    ).from_apply_func(triplestrend)
    


In [2]:
params = {
    'backtest': {
        'start': '2017-10-1',
        'end': '2018-1-1',
        'length': {
            'weeks': 4  #durée pendant laquelle on optimise les paramètres
        },
        'forward': {
            'weeks': 2  #durée pendant laquelle on joue les paramètres optimisés
        },
        
    },
    'strat':{
        'download_params': {
            'symbols': 'BTCUSDT',
            'interval': '1h',            
        },
        'get_params': {
            'column': ['High', 'Low', 'Close']
        },
        'run_params': { 
            'ema_window': {'start': 85, 'stop': 87, 'step': 1},
            'stl1': {'start': 15, 'stop': 17, 'step': 1},
            'stm1': {'start': 2, 'stop': 4, 'step': 1},
            'stl2': {'start': 15, 'stop': 17, 'step': 1},
            'stm2': {'start': 3, 'stop': 5, 'step': 1},
            'stl3': {'start': 35, 'stop': 37, 'step': 1},
            'stm3': {'start': 7, 'stop': 9, 'step': 1},
            'param_product': 'True'
        }, 
    },
    'portfolio': {'freq': 'H' ,
                  'fees' : 0.1/100,
                  'size_type': 'Percent',
                  'size': .1} #proportion du portefeuille à investir dans chaque trade
}



In [5]:
indics = Indicators()
test = ForwardTest(indics.TripleSTrend, params)
test.backtest(init_cash=1000)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-10-29 00:00:00+00:00 and 2017-11-12 00:00:00+00:00 finished at 2023-07-17 08:39:55.656116
Cash: 987.145918849611


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Forward test between 2017-12-10 00:00:00+00:00 and 2017-12-24 00:00:00+00:00 finished at 2023-07-17 08:41:44.585863
Cash: 978.2665513684088


0it [00:00, ?it/s]

978.2665513684088

In [ ]:
pf = test.aggregate_results ()
pf.stats()

Start                         2017-10-29 00:00:00+00:00
End                           2017-12-23 23:00:00+00:00
Period                                 56 days 00:00:00
Start Value                                       100.0
End Value                                     99.548976
Total Return [%]                              -0.451024
Benchmark Return [%]                          133.38779
Max Gross Exposure [%]                        10.942965
Total Fees Paid                                0.030488
Max Drawdown [%]                               2.382989
Max Drawdown Duration                  36 days 23:00:00
Total Trades                                          2
Total Closed Trades                                   1
Total Open Trades                                     1
Open Trade PnL                                 -0.90357
Win Rate [%]                                      100.0
Best Trade [%]                                 4.529978
Worst Trade [%]                                4